## Simulate Gain in a Ti:Sapphire Laser
The laser pulse duration $T_{laser}$ is typically 200 ps to 1 ns, while the Ti-Sapphire phase relaxation time $T_{crystal}$ is between $10 fs$ and $10 ps$.
<br>Because $T_{crystal} << T_{laser}$, we can use the Frantz-Nodvik (F-N) rate equations.
<br> This approach can be used for other crystals as well.
<br>
***
| The Frantz-Nodvik (F-N) rate equations:                                                        |     |
| :----------------------------------------------------------------------------------------------| :---|
| $\frac{d}{dt}\Delta_{ts}(t) = \gamma_{ts} \sigma_{ts} c n_{ph}(t) \Delta_{ts}(t)$                      | (1) |
| $\left( \frac{d}{dt} + \frac{d}{dz} \right) n_{ph}(t) = \sigma_{ts} c n_{ph}(t) \Delta_{ts}(t)$ | (2) |
<br>
In the next cell, we force the tables in this notebook to be left justified. (The default is center justified, which is annoying.)

In [1]:
%%html
<style>
table {float:left}
</style>

#### Dynamical equations for laser amplification in a Ti-Sapphire crystal --
***
| Quantity                 | Units            | Variable Name      | Description                                                            |
| :----------------------: | :--------------: | :---------------:  | :--------------------------------------------------------------------: |
| $\Delta_{ts}(t)$         | $cm^{-3}$        | $\delta_{ts}$      | Number density of inverted states                                      |
| $n_{ph}(t)$              | $cm^{-3}$        | $n_{ph}$           | Number density of infrared photons propagating through the crystal     |
| $\gamma_{ts}$            |     TBD          | gamma_ts           | $\gamma_{ts} = 1 + \gamma_{ts,1} / \gamma_{ts,2}$                      |
| $\gamma_{ts,1}$          |     TBD          | gamma_ts_1         | Degeneracy of the lower energy levels                                  |
| $\gamma_{ts,2}$          |     TBD          | gamma_ts_2         | Degeneracy of the upper energy levels                                  |
| $\sigma_{ts}$            | $cm^2$           | sigma_ts           | Ti-Sapphire emission cross-section                                     |
| $c$                      | $cm/s$            | c                 | Speed of light                                                         |

In [2]:
# Specify initial conditions for relevant variables
gamma_ts_1 = 1.
gamma_ts_2 = 1.
gamma_ts = 1 + gamma_ts_1 / gamma_ts_2

sigma_ts = 1.5e-19  # at lambda_0 = 800 nm

delta_ts_0 = 1.e21
n_ph_0 = 1.e11

#### Approximate analytical solution
The F-N eqn.'s can be solved semi-analytically, involving quantities 
to be obtained via quadrature, given two assumptions --
1. The seed laser pulse is square, with a full width of $\tau_{pulse}$.
2. $\Delta_{ts}(t=0)$ is independent of longitudinal position $z$.

| The approximate F-N rate equation dynamics:                                                      |   |
| :------------------------------------------------------------------------------------------------|:--|
| $n_{ph}(\vec{x},t) = \frac{n_0}{1 - \big[ 1 - \exp \big(-\sigma_{ts} \Delta_0 x \big) \big] \exp \big[ -\gamma_{ts}  \sigma_{ts}  \eta_{pulse} (t - z/c) \big/ \tau_{pulse} \big] }$                                                                                      |(3)|
| $n_0 \left[ cm^{-3} \right]$ is the photon number density associated with the square laser pulse |(4)|
| $\Delta_0 \left[ cm^{-3} \right]$ is the (assumed to be uniform) inversion density               |(5)|
| $\eta_{pulse} = n_0  c  \tau_{pulse} \left[ cm^{-2} \right]$                                     |(6)|

|For low-power pulses and low gain, integrate the above eqn to infer $G_E$, the energy gain:| |
|:-------------------------------------------------------------------------------------|:---|
|$G_E = \ln \left\{ 1 + \big[ \exp(\gamma_{ts} \sigma_{ts} \eta_{pulse}) - 1 \big] \exp(\gamma_0  \sigma_{ts}  L_{ts}) \right\} \ \big/ \ (\gamma_{ts}  \sigma_{ts}  \eta_{pulse})$|(8)|
|where $L_{ts}$ is the length of the crystal |  |

In [3]:
# We can't do anything without Numpy and the speed of light...
import numpy as np
from scipy.constants import c
c_cm = 100.*c

# More initial conditions:
tau_pulse = 1.e-9
eta_pulse = n_ph_0 * c_cm * tau_pulse
length_ts = 1.  # length of crystal [cm]

epsilon_ts = gamma_ts*sigma_ts*eta_pulse
print('  eta_pulse = ', eta_pulse)
print(' epsilon_ts = ', epsilon_ts)

g_e_max = 1. + delta_ts_0*length_ts/gamma_ts/eta_pulse
factor_ts = sigma_ts*delta_ts_0*length_ts
# factor_max = np.log((np.exp(epsilon_ts*g_e_max)-1.)/(np.exp(g_e_max)-1.))
factor_max = np.log(g_e_max)
print('  factor_ts = ', factor_ts)
print(' factor_max = ', factor_max)


# Analytic solution for photon amplification factor, assuming delta_ts is uniform
# Averaged value for the number of inverted states (assuming the function is flattened)
g_e = g_e_max
delta_ts = 0.
if factor_ts < factor_max:
    g_e = np.log(1.+np.exp(factor_ts)*(np.exp(epsilon_ts)-1.))/epsilon_ts
    delta_ts = delta_ts_0 - (gamma_ts*eta_pulse/length_ts) * (g_e - 1.)

n_ph = g_e * n_ph_0

print('delta_ts_0 = ', delta_ts_0)
print('  delta_ts = ', delta_ts)
print('    n_ph_0 = ', n_ph_0)
print('       g_e = ', g_e)
print('      n_ph = ', n_ph)

  eta_pulse =  2997924580000.0
 epsilon_ts =  8.99377374e-07
  factor_ts =  150.0
 factor_max =  18.932198419789525
delta_ts_0 =  1e+21
  delta_ts =  0.0
    n_ph_0 =  100000000000.0
       g_e =  166782048.59907603
      n_ph =  1.6678204859907602e+19


### Plotting

In [6]:
import matplotlib.pyplot as plt
from matplotlib import ticker, colors
%matplotlib widget


# Analytic solution for resulting profile of inverted states
num_points = 100
delta_ts_1 = np.zeros(num_points)
for i in range(num_points):
    x = i*length_ts/(num_points-1)
    delta_ts_1[i] = delta_ts_0 / (1.+np.exp(sigma_ts*delta_ts_0*x)*(np.exp(epsilon_ts)-1.))

plt.figure()

plt.title('Laser-pulse amplification', fontsize=16)
plt.xlabel('Propagation inside the crystal [m]', fontsize=16)
plt.ylabel(r'$n_{ph}\ [cm^{-3}]$', fontsize=16)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …